In [45]:
# work with saved historic data to minimize the number of api requests

import pandas as pd
import os

symbol = 'btc'

#tick_data = pd.read_csv('online_trading_data.csv', index_col=1, infer_datetime_format=True, parse_dates=True)

tick_data = pd.read_csv(symbol+'_daily.v.txt', index_col=2, infer_datetime_format=True, parse_dates=True)

#tick_data.iloc[:, 0] = pd.to_datetime(tick_data.iloc[:, 0], infer_datetime_format=True, unit='s')

#tick_data.set_index('time', inplace=True)

tick_data.sort_index(inplace=True)

# implement re-sampling
'''
bar lengths are in seconds
5 min: 300s, 10 min: 600s, 20 min: 1200s, 40 min: 2400s, 60min: 3600s
3 hr: 10800s, 6hr: 21600s, 9hr: 32400s, 12hr: 43200s
'''
df = tick_data['<CLOSE>']
#bar = '300s'
#df = tick_data.resample(bar, label='right').last().ffill()
df.head()
pd.plotting.autocorrelation_plot(df[-60:])

<AxesSubplot:xlabel='Lag', ylabel='Autocorrelation'>

In [53]:
import time
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

sub = pd.DataFrame(df)

# analyze data sub-set

i = 2 # Number of ticks to use for simple moving average (SMA)
t = str(time.time()) #datetime stamp

sub['SMA'] = sub['<CLOSE>'].rolling(i).mean().shift(i)
sub.plot(figsize=(10,6), lw=0.75, title='BTC-USD', ylabel='USD/BTC')
#plt.savefig('1-BTC-USD_SMA_{}.png'.format(t))
plt.show()
sub.head()

,<CLOSE>,SMA
<DATE>,,
2010-07-17,0.04951,NaN
2010-07-18,0.08584,NaN
2010-07-19,0.08080,NaN
2010-07-20,0.07474,0.067675
2010-07-21,0.07921,0.083320


In [54]:
import pandas as pd
import numpy as np

# vectorized backtesting

ptc = 0.005 # proportional transactional costs - Coinbase Pro charges 0.5% per transaction

data = pd.DataFrame()
data['close'] = sub['<CLOSE>']
data['returns'] = np.log(data['close']/data['close'].shift(1))

data.dropna(inplace=True)

data = data[-60:]

lags = 5

cols = []

for lag in range(1, lags+1):
    col = 'lag_{}'.format(lag)
    data[col] = data['returns'].shift(lag)
    cols.append(col)

data.dropna(inplace=True)

data[cols] = np.where(data[cols] >= 0, 1, 0)

data['direction'] = np.where(data['returns'] > 0, 1, -1)

data[cols + ['direction']].head()

,lag_1,lag_2,lag_3,lag_4,lag_5,direction
<DATE>,,,,,,
2020-11-16,0.0,0.0,1.0,1.0,1.0,1
2020-11-17,1.0,0.0,0.0,1.0,1.0,1
2020-11-18,1.0,1.0,0.0,0.0,1.0,1
2020-11-19,1.0,1.0,1.0,0.0,0.0,1
2020-11-20,1.0,1.0,1.0,1.0,0.0,1


In [39]:
data.to_csv('backtest_data.csv')

In [55]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from datetime import datetime
import pickle

model = SVC(C=1, kernel='linear', gamma='auto')

split = int(len(data) * 0.80)

train = data.iloc[:split].copy()

model.fit(train[cols], train['direction'])

# persisting the model object
t = str(time.time()) #datetime stamp
pickle.dump(model, open('algorithm-{}.pkl'.format(t), 'wb'))

In [56]:
accuracy_score(train['direction'], model.predict(train[cols]))

0.6818181818181818

In [57]:
test = data.iloc[split:].copy()

test['position'] = model.predict(test[cols])
#test['position'] = test['direction']         # naive prediction

test[['position','direction']].tail()

,position,direction
<DATE>,,
2021-01-05,1,1
2021-01-06,1,1
2021-01-07,1,1
2021-01-08,1,1
2021-01-09,1,-1


In [58]:
accuracy_score(test['direction'], test['position'])

0.8181818181818182

In [59]:
test['strategy'] = test['position'] * test['returns']
sum(test['position'].diff() != 0)

1

In [60]:
test['strategy_tc'] = np.where(test['position'].diff() != 0, test['strategy'] - ptc, test['strategy'])

In [61]:
test[['returns', 'strategy', 'strategy_tc']].sum().apply(np.exp)

returns        1.497012
strategy       1.497012
strategy_tc    1.489546
dtype: float64

In [62]:
t = str(time.time())
test[['returns', 'strategy', 'strategy_tc']].cumsum().apply(np.exp).plot(figsize=(10,6))
plt.savefig('1-BTC-USD_StratCompare_{}.png'.format(t))
plt.show()

In [20]:
import pandas as pd
from datetime import datetime
t = str(time.time()) #datetime stamp #datetime stamp
test.to_csv('backtest_{}.csv'.format(t))